# Skip-gram word2vec

In this notebook, I'll lead you through using TensorFlow to implement the word2vec algorithm using the skip-gram architecture. By implementing this, you'll learn about embedding words for use in natural language processing. This will come in handy when dealing with things like machine translation.

## Readings

Here are the resources I used to build this notebook. I suggest reading these either beforehand or while you're working on this material.

* A really good [conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) of word2vec from Chris McCormick 
* [First word2vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
* [NIPS paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for word2vec also from Mikolov et al.
* An [implementation of word2vec](http://www.thushv.com/natural_language_processing/word2vec-part-1-nlp-with-deep-learning-with-tensorflow-skip-gram/) from Thushan Ganegedara
* TensorFlow [word2vec tutorial](https://www.tensorflow.org/tutorials/word2vec)

## Word embeddings

When you're dealing with words in text, you end up with tens of thousands of classes to predict, one for each word. Trying to one-hot encode these words is massively inefficient, you'll have one element set to 1 and the other 50,000 set to 0. The matrix multiplication going into the first hidden layer will have almost all of the resulting values be zero. This a huge waste of computation. 

![one-hot encodings](assets/one_hot_encoding.png)

To solve this problem and greatly increase the efficiency of our networks, we use what are called embeddings. Embeddings are just a fully connected layer like you've seen before. We call this layer the embedding layer and the weights are embedding weights. We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix. We can do this because the multiplication of a one-hot encoded vector with a matrix returns the row of the matrix corresponding the index of the "on" input unit.

![lookup](assets/lookup_matrix.png)

Instead of doing the matrix multiplication, we use the weight matrix as a lookup table. We encode the words as integers, for example "heart" is encoded as 958, "mind" as 18094. Then to get hidden layer values for "heart", you just take the 958th row of the embedding matrix. This process is called an **embedding lookup** and the number of hidden units is the **embedding dimension**.

<img src='assets/tokenize_lookup.png' width=500>
 
There is nothing magical going on here. The embedding lookup table is just a weight matrix. The embedding layer is just a hidden layer. The lookup is just a shortcut for the matrix multiplication. The lookup table is trained just like any weight matrix as well.

Embeddings aren't only used for words of course. You can use them for any model where you have a massive number of classes. A particular type of model called **Word2Vec** uses the embedding layer to find vector representations of words that contain semantic meaning.



## Word2Vec

The word2vec algorithm finds much more efficient representations by finding vectors that represent the words. These vectors also contain semantic information about the words. Words that show up in similar contexts, such as "black", "white", and "red" will have vectors near each other. There are two architectures for implementing word2vec, CBOW (Continuous Bag-Of-Words) and Skip-gram.

<img src="assets/word2vec_architectures.png" width="500">

In this implementation, we'll be using the skip-gram architecture because it performs better than CBOW. Here, we pass in a word and try to predict the words surrounding it in the text. In this way, we can train the network to learn representations for words that show up in similar contexts.

First up, importing packages.

In [ ]:
import time

import numpy as np
import tensorflow as tf

import utils

Load the [text8 dataset](http://mattmahoney.net/dc/textdata.html), a file of cleaned up Wikipedia articles from Matt Mahoney. The next cell will download the data set to the `data` folder. Then you can extract it and delete the archive file to save storage space.

In [ ]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

dataset_folder_path = 'data'
dataset_filename = 'text8.zip'
dataset_name = 'Text8 Dataset'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(dataset_filename):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc=dataset_name) as pbar:
        urlretrieve(
            'http://mattmahoney.net/dc/text8.zip',
            dataset_filename,
            pbar.hook)

if not isdir(dataset_folder_path):
    with zipfile.ZipFile(dataset_filename) as zip_ref:
        zip_ref.extractall(dataset_folder_path)
        
with open('data/text8') as f:
    text = f.read()

## Preprocessing

Here I'm fixing up the text to make training easier. This comes from the `utils` module I wrote. The `preprocess` function coverts any punctuation into tokens, so a period is changed to ` <PERIOD> `. In this data set, there aren't any periods, but it will help in other NLP problems. I'm also removing all words that show up five or fewer times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. If you want to write your own functions for this stuff, go for it.

In [ ]:
words = utils.preprocess(text)
print(words[:30])

In [ ]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

And here I'm creating dictionaries to convert words to integers and backwards, integers to words. The integers are assigned in descending frequency order, so the most frequent word ("the") is given the integer 0 and the next most frequent is 1 and so on. The words are converted to integers and stored in the list `int_words`.

In [ ]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

## Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

I'm going to leave this up to you as an exercise. This is more of a programming challenge, than about deep learning specifically. But, being able to prepare your data for your network is an important skill to have. Check out my solution to see how I did it.

> **Exercise:** Implement subsampling for the words in `int_words`. That is, go through `int_words` and discard each word given the probablility $P(w_i)$ shown above. Note that $P(w_i)$ is the probability that a word is discarded. Assign the subsampled data to `train_words`.

In [ ]:
from collections import Counter
import math

counter = Counter(words)
vocab = list(counter.keys())
P_vocab = Counter()
threshold = 100000

for key, val in counter.items():
    P_vocab[key] = 1 - math.sqrt(threshold / val)

In [ ]:
train_words = []
remove_list = []

for key, val in P_vocab.items():
    if val > 0:
        remove_list.append(key)
        
        

In [ ]:
train_words = [word for word in words if not word in remove_list]
vocab = [word for word in vocab if not word in remove_list]

In [ ]:
vocab_to_int, int_to_vocab = []
int_words = [vocab_to_int[word] for word in words]

In [ ]:
train_counter = Counter(train_words)
vocab_to_int = {word: key for key, (word, count) in enumerate(train_counter.most_common())}
int_to_vocab = {key: word for key, (word, count) in enumerate(train_counter.most_common())}

In [ ]:
train_words = [vocab_to_int[word] for word in train_words]

## Making batches

Now that our data is in good shape, we need to get it into the proper form to pass it into our network. With the skip-gram architecture, for each word in the text, we want to grab all the words in a window around that word, with size $C$. 

From [Mikolov et al.](https://arxiv.org/pdf/1301.3781.pdf): 

"Since the more distant words are usually less related to the current word than those close to it, we give less weight to the distant words by sampling less from those words in our training examples... If we choose $C = 5$, for each training word we will select randomly a number $R$ in range $< 1; C >$, and then use $R$ words from history and $R$ words from the future of the current word as correct labels."

> **Exercise:** Implement a function `get_target` that receives a list of words, an index, and a window size, then returns a list of words in the window around the index. Make sure to use the algorithm described above, where you choose a random number of words from the window.

In [ ]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    # Your code here
    target = []
    for index in range(idx - window_size, idx + window_size + 1):
        if index > 0 and index < len(words) and index != idx:
            target.append(words[index])
    return target

Here's a function that returns batches for our network. The idea is that it grabs `batch_size` words from a words list. Then for each of those words, it gets the target words in the window. I haven't found a way to pass in a random number of target words and get it to work with the architecture, so I make one row per input-target pair. This is a generator function by the way, helps save memory.

In [ ]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y
    

## Building the graph

From [Chris McCormick's blog](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/), we can see the general structure of our network.
![embedding_network](./assets/skip_gram_net_arch.png)

The input words are passed in as integers. This will go into a hidden layer of linear units, then into a softmax layer. We'll use the softmax layer to make a prediction like normal.

The idea here is to train the hidden layer weight matrix to find efficient representations for our words. We can discard the softmax layer becuase we don't really care about making predictions with this network. We just want the embedding matrix so we can use it in other networks we build from the dataset.

I'm going to have you build the graph in stages now. First off, creating the `inputs` and `labels` placeholders like normal.

> **Exercise:** Assign `inputs` and `labels` using `tf.placeholder`. We're going to be passing in integers, so set the data types to `tf.int32`. The batches we're passing in will have varying sizes, so set the batch sizes to [`None`]. To make things work later, you'll need to set the second dimension of `labels` to `None` or `1`.

In [ ]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, shape=(None))
    labels = tf.placeholder(tf.float32, shape=(None, None))

## Embedding



The embedding matrix has a size of the number of words by the number of units in the hidden layer. So, if you have 10,000 words and 300 hidden units, the matrix will have size $10,000 \times 300$. Remember that we're using tokenized data for our inputs, usually as integers, where the number of tokens is the number of words in our vocabulary.


> **Exercise:** Tensorflow provides a convenient function [`tf.nn.embedding_lookup`](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup) that does this lookup for us. You pass in the embedding matrix and a tensor of integers, then it returns rows in the matrix corresponding to those integers. Below, set the number of embedding features you'll use (200 is a good start), create the embedding matrix variable, and use `tf.nn.embedding_lookup` to get the embedding tensors. For the embedding matrix, I suggest you initialize it with a uniform random numbers between -1 and 1 using [tf.random_uniform](https://www.tensorflow.org/api_docs/python/tf/random_uniform).

In [ ]:
n_vocab = len(int_to_vocab)
n_embedding =  200
with train_graph.as_default():
    embedding = tf.Variable(tf.random_uniform(shape=(n_vocab, n_embedding)))
    embed = tf.nn.embedding_lookup(embedding, inputs)

## Negative sampling



For every example we give the network, we train it using the output from the softmax layer. That means for each input, we're making very small changes to millions of weights even though we only have one true example. This makes training the network very inefficient. We can approximate the loss from the softmax layer by only updating a small subset of all the weights at once. We'll update the weights for the correct label, but only a small number of incorrect labels. This is called ["negative sampling"](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf). Tensorflow has a convenient function to do this, [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss).

> **Exercise:** Below, create weights and biases for the softmax layer. Then, use [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss) to calculate the loss. Be sure to read the documentation to figure out how it works.

In [ ]:
# Number of negative labels to sample
n_sampled = 100
with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal((n_vocab, n_embedding), stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(n_vocab))
    
    # Calculate the loss using negative sampling
    loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, 
                                      labels, embed,
                                      n_sampled, n_vocab)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

## Validation

This code is from Thushan Ganegedara's implementation. Here we're going to choose a few common words and few uncommon words. Then, we'll print out the closest words to them. It's a nice way to check that our embedding table is grouping together words with similar semantic meanings.

In [ ]:
import random
with train_graph.as_default():
    ## From Thushan Ganegedara's implementation
    valid_size = 16 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

In [ ]:
# If the checkpoints directory doesn't exist:
!mkdir checkpoints

## Training

Below is the code to train the network. Every 100 batches it reports the training loss. Every 1000 batches, it'll print out the validation words.

In [76]:
epochs = 10
batch_size = 1000
window_size = 10

with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:
            
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            if iteration % 1000 == 0:
                ## From Thushan Ganegedara's implementation
                # note that this is expensive (~20% slowdown if computed every 500 steps)
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
    save_path = saver.save(sess, "checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

Epoch 2/10 Iteration: 17000 Avg. Training loss: 4.5389 0.0246 sec/batch
Nearest to will: can, should, must, would, may, might, only, it,
Nearest to on: this, from, with, which, an, while, all, after,
Nearest to name: names, word, from, derived, meaning, another, means, comes,
Nearest to after: before, later, during, when, following, first, which, while,
Nearest to who: those, while, their, them, his, they, had, being,
Nearest to he: his, him, himself, later, after, had, returned, moved,
Nearest to more: less, rather, much, than, even, better, other, some,
Nearest to under: over, which, later, with, after, following, control, only,
Nearest to http: org, www, com, htm, html, edu, forum, links,
Nearest to pre: during, modern, widely, first, often, today, ancient, known,
Nearest to start: before, end, up, after, back, when, time, last,
Nearest to older: age, females, median, size, household, males, families, poverty,
Nearest to troops: forces, army, battle, allied, invasion, soldiers, with

Epoch 2/10 Iteration: 21100 Avg. Training loss: 4.3061 0.0278 sec/batch
Epoch 2/10 Iteration: 21200 Avg. Training loss: 4.4057 0.0264 sec/batch
Epoch 2/10 Iteration: 21300 Avg. Training loss: 4.5413 0.0262 sec/batch
Epoch 2/10 Iteration: 21400 Avg. Training loss: 4.3759 0.0249 sec/batch
Epoch 2/10 Iteration: 21500 Avg. Training loss: 4.4822 0.0249 sec/batch
Epoch 2/10 Iteration: 21600 Avg. Training loss: 4.5011 0.0274 sec/batch
Epoch 2/10 Iteration: 21700 Avg. Training loss: 4.4487 0.0264 sec/batch
Epoch 2/10 Iteration: 21800 Avg. Training loss: 4.4757 0.0265 sec/batch
Epoch 2/10 Iteration: 21900 Avg. Training loss: 4.5380 0.0265 sec/batch
Epoch 2/10 Iteration: 22000 Avg. Training loss: 4.5696 0.0273 sec/batch
Nearest to will: can, must, would, should, might, shall, only, may,
Nearest to on: which, this, with, an, from, while, made, another,
Nearest to name: names, named, changed, from, word, meaning, originally, comes,
Nearest to after: before, later, when, during, following, first, w

Epoch 3/10 Iteration: 25800 Avg. Training loss: 4.4558 0.0244 sec/batch
Epoch 3/10 Iteration: 25900 Avg. Training loss: 4.4273 0.0238 sec/batch
Epoch 3/10 Iteration: 26000 Avg. Training loss: 4.4794 0.0260 sec/batch
Nearest to will: can, must, should, would, may, cannot, might, shall,
Nearest to on: while, which, upon, both, this, from, with, only,
Nearest to name: names, changed, named, word, meaning, originally, another, nickname,
Nearest to after: before, later, when, following, during, first, which, while,
Nearest to who: those, while, whom, them, whose, they, their, but,
Nearest to he: his, him, himself, later, it, father, returned, went,
Nearest to more: less, rather, much, better, even, than, far, larger,
Nearest to under: over, which, with, later, control, only, from, following,
Nearest to http: www, org, htm, com, html, edu, gov, https,
Nearest to pre: during, first, era, modern, early, world, use, often,
Nearest to start: end, before, after, back, when, beginning, up, startin

Epoch 3/10 Iteration: 30100 Avg. Training loss: 4.4367 0.0276 sec/batch
Epoch 3/10 Iteration: 30200 Avg. Training loss: 4.3718 0.0251 sec/batch
Epoch 3/10 Iteration: 30300 Avg. Training loss: 4.4698 0.0267 sec/batch
Epoch 3/10 Iteration: 30400 Avg. Training loss: 4.1740 0.0248 sec/batch
Epoch 3/10 Iteration: 30500 Avg. Training loss: 4.3254 0.0253 sec/batch
Epoch 3/10 Iteration: 30600 Avg. Training loss: 4.2550 0.0245 sec/batch
Epoch 3/10 Iteration: 30700 Avg. Training loss: 4.3920 0.0256 sec/batch
Epoch 3/10 Iteration: 30800 Avg. Training loss: 4.4339 0.0243 sec/batch
Epoch 3/10 Iteration: 30900 Avg. Training loss: 4.4064 0.0248 sec/batch
Epoch 3/10 Iteration: 31000 Avg. Training loss: 4.5288 0.0257 sec/batch
Nearest to will: can, must, would, should, might, may, shall, cannot,
Nearest to on: which, while, this, upon, both, following, from, with,
Nearest to name: names, changed, nickname, named, originally, meaning, word, simply,
Nearest to after: before, later, following, when, durin

Epoch 4/10 Iteration: 34600 Avg. Training loss: 4.4053 0.0251 sec/batch
Epoch 4/10 Iteration: 34700 Avg. Training loss: 4.2152 0.0257 sec/batch
Epoch 4/10 Iteration: 34800 Avg. Training loss: 4.4016 0.0256 sec/batch
Epoch 4/10 Iteration: 34900 Avg. Training loss: 4.4293 0.0241 sec/batch
Epoch 4/10 Iteration: 35000 Avg. Training loss: 4.4250 0.0248 sec/batch
Nearest to will: can, must, would, should, may, might, able, shall,
Nearest to on: while, upon, both, which, this, only, from, side,
Nearest to name: names, changed, named, meaning, word, originally, nickname, simply,
Nearest to after: before, later, following, when, during, first, while, however,
Nearest to who: those, whose, while, whom, themselves, which, their, they,
Nearest to he: his, him, himself, it, later, returned, went, which,
Nearest to more: less, rather, much, better, even, than, other, larger,
Nearest to under: over, which, later, after, only, well, from, control,
Nearest to http: www, org, htm, com, edu, html, gov, g

...

Epoch 9/10 Iteration: 88700 Avg. Training loss: 4.2991 0.0250 sec/batch
Epoch 9/10 Iteration: 88800 Avg. Training loss: 4.2475 0.0256 sec/batch
Epoch 9/10 Iteration: 88900 Avg. Training loss: 4.2182 0.0254 sec/batch
Epoch 9/10 Iteration: 89000 Avg. Training loss: 4.2468 0.0254 sec/batch
Nearest to will: can, must, should, would, continue, may, cannot, might,
Nearest to on: upon, while, which, both, only, this, when, from,
Nearest to name: names, changed, meaning, named, nickname, simply, surname, naming,
Nearest to after: before, later, following, when, while, first, with, finally,
Nearest to who: whose, which, those, while, whom, them, when, however,
Nearest to he: his, himself, him, it, later, father, only, which,
Nearest to more: less, rather, much, better, larger, other, even, than,
Nearest to under: over, out, from, only, which, after, whose, at,
Nearest to http: www, htm, com, edu, org, html, gov, geocities,
Nearest to pre: era, during, period, prior, start, earlier, beginning, o

Epoch 9/10 Iteration: 93100 Avg. Training loss: 4.2715 0.0254 sec/batch
Epoch 9/10 Iteration: 93200 Avg. Training loss: 4.1911 0.0229 sec/batch
Epoch 9/10 Iteration: 93300 Avg. Training loss: 4.2948 0.0255 sec/batch
Epoch 9/10 Iteration: 93400 Avg. Training loss: 4.2787 0.0240 sec/batch
Epoch 9/10 Iteration: 93500 Avg. Training loss: 4.2417 0.0248 sec/batch
Epoch 9/10 Iteration: 93600 Avg. Training loss: 4.2802 0.0255 sec/batch
Epoch 9/10 Iteration: 93700 Avg. Training loss: 4.2257 0.0280 sec/batch
Epoch 9/10 Iteration: 93800 Avg. Training loss: 4.2584 0.0263 sec/batch
Epoch 9/10 Iteration: 93900 Avg. Training loss: 4.2211 0.0273 sec/batch
Epoch 9/10 Iteration: 94000 Avg. Training loss: 4.2844 0.0266 sec/batch
Nearest to will: can, should, may, must, would, continue, tend, might,
Nearest to on: upon, only, which, both, while, this, when, from,
Nearest to name: names, changed, surname, naming, simply, named, nickname, meaning,
Nearest to after: before, later, following, when, during, wh

Epoch 9/10 Iteration: 97800 Avg. Training loss: 4.2007 0.0256 sec/batch
Epoch 9/10 Iteration: 97900 Avg. Training loss: 4.2511 0.0240 sec/batch
Epoch 9/10 Iteration: 98000 Avg. Training loss: 4.3069 0.0242 sec/batch
Nearest to will: can, must, may, should, would, tend, might, continue,
Nearest to on: upon, while, only, which, both, later, this, when,
Nearest to name: names, changed, simply, nickname, naming, surname, named, originally,
Nearest to after: before, later, following, when, while, with, during, first,
Nearest to who: whose, which, those, while, them, whom, when, themselves,
Nearest to he: his, him, himself, it, later, father, career, only,
Nearest to more: less, rather, much, better, other, larger, even, far,
Nearest to under: over, out, from, which, only, whose, after, entire,
Nearest to http: www, htm, edu, com, html, org, gov, geocities,
Nearest to pre: era, during, period, prior, start, oldest, subsequent, end,
Nearest to start: beginning, end, begin, time, starting, sta

Epoch 10/10 Iteration: 102100 Avg. Training loss: 4.2826 0.0246 sec/batch
Epoch 10/10 Iteration: 102200 Avg. Training loss: 4.2130 0.0251 sec/batch
Epoch 10/10 Iteration: 102300 Avg. Training loss: 4.2319 0.0264 sec/batch
Epoch 10/10 Iteration: 102400 Avg. Training loss: 4.2343 0.0255 sec/batch
Epoch 10/10 Iteration: 102500 Avg. Training loss: 4.1103 0.0236 sec/batch
Epoch 10/10 Iteration: 102600 Avg. Training loss: 4.2324 0.0245 sec/batch
Epoch 10/10 Iteration: 102700 Avg. Training loss: 4.2866 0.0249 sec/batch
Epoch 10/10 Iteration: 102800 Avg. Training loss: 4.2782 0.0260 sec/batch
Epoch 10/10 Iteration: 102900 Avg. Training loss: 4.2608 0.0253 sec/batch
Epoch 10/10 Iteration: 103000 Avg. Training loss: 4.2817 0.0254 sec/batch
Nearest to will: can, must, should, may, continue, would, tend, might,
Nearest to on: upon, while, both, only, which, when, this, another,
Nearest to name: names, changed, simply, nickname, named, surname, meaning, word,
Nearest to after: before, later, follow

Epoch 10/10 Iteration: 106600 Avg. Training loss: 4.2997 0.0239 sec/batch
Epoch 10/10 Iteration: 106700 Avg. Training loss: 4.2808 0.0254 sec/batch
Epoch 10/10 Iteration: 106800 Avg. Training loss: 4.2954 0.0244 sec/batch
Epoch 10/10 Iteration: 106900 Avg. Training loss: 4.2596 0.0234 sec/batch
Epoch 10/10 Iteration: 107000 Avg. Training loss: 4.1717 0.0256 sec/batch
Nearest to will: can, may, should, must, would, continue, tend, might,
Nearest to on: which, only, upon, while, both, from, this, an,
Nearest to name: names, nickname, changed, surname, simply, naming, named, meaning,
Nearest to after: before, following, later, when, with, during, while, first,
Nearest to who: those, which, while, whose, them, themselves, whom, however,
Nearest to he: his, him, himself, it, later, career, she, them,
Nearest to more: less, rather, much, even, better, other, larger, far,
Nearest to under: over, out, from, which, after, whose, only, at,
Nearest to http: www, htm, com, edu, html, org, gov, geo

Restore the trained network if you need to:

In [ ]:
with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    embed_mat = sess.run(embedding)

## Visualizing the word vectors

Below we'll use T-SNE to visualize how our high-dimensional word vectors cluster together. T-SNE is used to project these vectors into two dimensions while preserving local stucture. Check out [this post from Christopher Olah](http://colah.github.io/posts/2014-10-Visualizing-MNIST/) to learn more about T-SNE and other ways to visualize high-dimensional data.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
viz_words = 500
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)